In [1]:
from tensorflow import keras  
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import json
import tensorflow_datasets as tfds

In [2]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


In [3]:
dataset = pd.read_csv("./train_data.csv")


In [4]:
dataset.columns=['index','Type','rValue','rRange','centerDistance']
dataset.head()

,index,Type,rValue,rRange,centerDistance
0,0,16,0,0.0000,59.0
1,1,120,0,0.0000,63.0
2,2,8,0,0.0000,298.0
3,3,16,0,0.0000,207.0
4,4,181,0,0.0004,37.0


In [5]:
y = dataset.loc[:,'rValue']
y.value_counts()

0     176737
10     42050
20     11467
30      1086
40       159
50        67
60         5
90         1
70         1
Name: rValue, dtype: int64

In [6]:
x = dataset.loc[:,['Type','centerDistance']]

In [7]:
x['centerDistance']=round(x['centerDistance']/3062,4)

In [8]:
x['Type']=round(x['Type']/216,2)

In [9]:
y = pd.DataFrame(y)

In [10]:
y

,rValue
0,0
1,0
2,0
3,0
4,0
...,...
231568,0
231569,0
231570,0
231571,10


In [11]:
y['rValue'] = round(y['rValue']/100,2)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [13]:
model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(100)
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [15]:
#简单的两层深度学习模型
#model.fit(x_train, y_train, epochs=10)

In [16]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(100)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
    return model

In [17]:
single_worker_model = get_compiled_model()

In [18]:
single_worker_model.fit(x_train, y_train, epochs=10)

Epoch 1/10
5428/5428 [==============================] - 4s 728us/step - loss: 0.1988 - accuracy: 0.7471
Epoch 2/10
5428/5428 [==============================] - 4s 713us/step - loss: 3.3618e-05 - accuracy: 0.7607
Epoch 3/10
5428/5428 [==============================] - 4s 701us/step - loss: 1.0094e-05 - accuracy: 0.7626
Epoch 4/10
5428/5428 [==============================] - 4s 704us/step - loss: 9.8947e-06 - accuracy: 0.7633
Epoch 5/10
5428/5428 [==============================] - 4s 707us/step - loss: 9.8944e-06 - accuracy: 0.7623
Epoch 6/10
5428/5428 [==============================] - 4s 702us/step - loss: 9.8944e-06 - accuracy: 0.7631
Epoch 7/10
5428/5428 [==============================] - 4s 704us/step - loss: 9.8944e-06 - accuracy: 0.7621
Epoch 8/10
5428/5428 [==============================] - 4s 717us/step - loss: 9.8944e-06 - accuracy: 0.7639
Epoch 9/10
5428/5428 [==============================] - 4s 710us/step - loss: 9.8944e-06 - accuracy: 0.7643
Epoch 10/10
5428/5428 [=========

In [19]:
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["192.168.1.102:20000", "192.168.1.10:20001"]
    },
    'task': {'type': 'worker', 'index': 0}
})

In [20]:
#multiworker_strategy = tf.distribute.MultiWorkerMirroredStrategy()
#挪到最前面来bypass

In [21]:
with strategy.scope():
    #model = make_or_restore_model()
    model.fit(x_train, y_train, epochs=5)

Epoch 1/5
5428/5428 [==============================] - 6s 935us/step - loss: 0.0547 - accuracy: 0.7554
Epoch 2/5
5428/5428 [==============================] - 5s 913us/step - loss: 1.0625e-05 - accuracy: 0.7622
Epoch 3/5
5428/5428 [==============================] - 5s 929us/step - loss: 1.0057e-05 - accuracy: 0.7631
Epoch 4/5
5428/5428 [==============================] - 5s 929us/step - loss: 9.8944e-06 - accuracy: 0.7636
Epoch 5/5
5428/5428 [==============================] - 5s 957us/step - loss: 9.8944e-06 - accuracy: 0.7625
